In [2]:
from owlready2 import *


In [3]:
#PascalCase
def to_pascal_case(text):
    text = text.replace('_', ' ').replace('-', ' ')
    words = text.split()
    return ''.join([word.capitalize() for word in words])
#camelCase
def to_camel_case(prop_name):
    words = prop_name.replace("_", " ").split()
    return words[0].lower() + "".join(word.capitalize() for word in words[1:]) if words else ""

In [140]:
#rdf to puml
def rdf_to_puml(input_rdf, output_puml):
    data = get_ontology(input_rdf).load()
    iofcore = get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
    bfo = get_namespace("http://purl.obolibrary.org/obo/")
    
    classes = {}
    individuals = {}
    properties = []

    for ind in data.individuals():
        individuals[ind.name] = ind
        if ind.is_a:
            for ind_type in ind.is_a:
                try:
                    ns = bfo if ind_type.name.startswith("BFO") else iofcore
                    class_label = to_pascal_case(ns[ind_type.name].label[0])
                    classes[class_label] = ind_type
                    properties.append((ind, "typeOf", class_label))
                except:
                    continue
        
        for prop in ind.get_properties():
            for value in prop[ind]:
                ns = bfo if prop.name.startswith("BFO") else iofcore
                properties.append((ind, to_camel_case(ns[prop.name].label[0]), value))
    print('classes',classes,'\n', 'individuals',individuals,'\n', properties)

    with open(output_puml, "w") as f:
        f.write("@startuml\n")
        f.write("!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml\n")

        # Declare classes first with appropriate prefixes
        class_map = {}
        for idx, (cls_label, cls) in enumerate(classes.items(), start=1):
            ns_prefix = "bfo:" if cls.name.startswith("BFO") else "iof:"
            formatted_label = f"{ns_prefix}{cls_label}"
            class_map[cls_label] = f"c{idx}"
            f.write(f"class({class_map[cls_label]}, {formatted_label})\n")

        # Declare individuals with ns1: prefix
        individual_map = {}
        for idx, (ind_name, ind) in enumerate(individuals.items(), start=1):
            individual_map[ind_name] = f"i{idx}"
            f.write(f"individual({individual_map[ind_name]}, ns1:{ind_name})\n")

        # Add typeOf relationships
        for s, p, o in properties:
            if p == "typeOf" and o in class_map:
                f.write(f"typeOf({individual_map[s.name]}, {class_map[o]})\n")

        # Add properties
        for s, p, o in properties:
            if p != "typeOf" and o.name in individual_map:
                f.write(f"property({individual_map[s.name]}, {p}, {individual_map[o.name]})\n")

        f.write("@enduml\n")

In [141]:
if __name__ == "__main__":
    input_rdf = "../sample/object-graph-1.rdf" 
    output_puml = "output1.puml"
    rdf_to_puml(input_rdf, output_puml)
    print(f"PUML file saved as {output_puml}")


classes {'MaterialArtifact': Core.MaterialArtifact, 'Process': obo.BFO_0000015, 'Quality': obo.BFO_0000019} 
 individuals {'artifact1': object-graph-1.artifact1, 'process1': object-graph-1.process1, 'length1': object-graph-1.length1} 
 [(object-graph-1.artifact1, 'typeOf', 'MaterialArtifact'), (object-graph-1.artifact1, 'participatesIn', object-graph-1.process1), (object-graph-1.artifact1, 'bearerOf', object-graph-1.length1), (object-graph-1.process1, 'typeOf', 'Process'), (object-graph-1.process1, 'hasParticipant', object-graph-1.artifact1), (object-graph-1.length1, 'typeOf', 'Quality'), (object-graph-1.length1, 'inheresIn', object-graph-1.artifact1)]
PUML file saved as output1.puml


In [ ]:
#Axiom to PUML
from owlready2 import *

def axiom_to_puml(class_entity):
    bfo = get_namespace("http://purl.obolibrary.org/obo/")
    iofcore = get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
    """Converts an OWL equivalent class axiom to PUML notation."""
    puml_output = []
    puml_output.append("!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml")
    class_map = {}
    counter = 1

    def get_class_name(entity):
        """Extracts and maps class names with proper prefixes."""
        nonlocal counter
        if isinstance(entity, ThingClass):
            print(counter, 'processing', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            class_name = entity.name
            if class_name not in class_map:
                class_map[class_name] = f"c{counter}"
                puml_output.append(f'class({class_map[class_name]}, {prefix}{to_camel_case(bfo[entity.name].label[0]) if class_name.startswith("BFO") else class_name})')
                counter += 1
            print(puml_output)
            return class_map[class_name]

        elif isinstance(entity, Restriction):
            if isinstance(entity.property, Inverse):    
                print(counter)
                print('Restriction', entity)
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(bfo[entity.property.property.name].label[0]) if entity.property.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
            else:
                print(counter)
                print('Restriction', entity)
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(bfo[entity.property.name].label[0]) if entity.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
        
        elif isinstance(entity, And): 
            print('And', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Or): 
            print('Or', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Not):
            return 
        
        elif isinstance(entity, Inverse):
            print('Inverse', entity.property)
            return

        else:
            print('error',entity)
            raise ValueError(f"Unsupported entity type: {type(entity)}")

    def process_restriction(restriction, prop_name):
        """Handles OWL 'some' restrictions including unions inside."""
        nonlocal counter
        if isinstance(restriction, And):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"intersection({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        
        elif isinstance(restriction, Or):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        

        elif isinstance(restriction.value, Or):
            print('or', restriction.value)
            entities = [get_class_name(e) for e in restriction.value.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities) 
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name

        else:  
            print('restriction else', restriction)
            class_name = get_class_name(restriction.value)
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {class_name})")
            counter += 1
            print(puml_output)
            return var_name
    
    if len(class_entity.equivalent_to) == 0:
        print('class_entity.equivalent_to',class_entity.equivalent_to)
        return
        
    else:
        print(class_entity.equivalent_to, type(class_entity.equivalent_to[0]))
        axiom = class_entity.equivalent_to[0]
        main_class_name = get_class_name(class_entity)
        processed_parts = [get_class_name(part) for part in axiom.Classes]
        #change from classes to axiom and let the func figure it out
        print(processed_parts)

        if len(processed_parts) > 1:
            final_union = f"ce{counter}"
            entity_list = ", ".join(f'"{e}"' for e in processed_parts)
            puml_output.append(f"union({final_union}, '[{entity_list}]')")
            puml_output.append(f"equivalent({main_class_name}, {final_union})")

        return "\n".join(puml_output)

# Example Usage: [ProcuringBusinessProcess,MaterialArtifact, PieceOfEquipment], [MeasurementInformationContentEntity]
input = "GainOfRole"
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
# axiom_result = axiom_to_puml(iofonto["ProcuringBusinessProcess"])
axiom_result = axiom_to_puml(iofonto[input])
print(axiom_result)


[obo.BFO_0000015 & (obo.BFO_0000199.some(obo.BFO_0000202 & (Core.meets.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyOverlaps.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyStarts.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)))) & Core.hasOutput.some(obo.BFO_0000023)) & obo.BFO_0000167.some(obo.BFO_0000004 & Not(obo.BFO_0000006))] <class 'owlready2.class_construct.And'>
1 processing Core.GainOfRole
['!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml', 'class(c1, iof:GainOfRole)']


TypeError: 'And' object is not iterable

In [220]:
print(iofonto["ProductProductionProcess"])
axiom = iofonto["ProductProductionProcess"].equivalent_to[0]
print(axiom.Classes)
print(type(axiom.Classes[0]), axiom.Classes[0])
print(type(axiom.Classes[1]), axiom.Classes[1])
print(type(axiom.Classes[2]), axiom.Classes[2])
print(type(axiom.Classes[1].value), axiom.Classes[1].value)
print(type(axiom.Classes[2].value), axiom.Classes[2].value)
print(type(axiom.Classes[2].value.Classes), axiom.Classes[2].value.Classes)
print(type(axiom.Classes[2].value.Classes[0]), axiom.Classes[2].value.Classes[0])


Core.ProductProductionProcess
[Core.BusinessProcess, obo.BFO_0000117.some(Core.ManufacturingProcess), obo.BFO_0000199.some(obo.BFO_0000008 & Core.temporallyOverlaps.some(Inverse(obo.BFO_0000108).some(Core.MaterialProduct))), Core.hasSpecifiedOutput.some(Core.MaterialProduct)]
<class 'owlready2.entity.ThingClass'> Core.BusinessProcess
<class 'owlready2.class_construct.Restriction'> obo.BFO_0000117.some(Core.ManufacturingProcess)
<class 'owlready2.class_construct.Restriction'> obo.BFO_0000199.some(obo.BFO_0000008 & Core.temporallyOverlaps.some(Inverse(obo.BFO_0000108).some(Core.MaterialProduct)))
<class 'owlready2.entity.ThingClass'> Core.ManufacturingProcess
<class 'owlready2.class_construct.And'> obo.BFO_0000008 & Core.temporallyOverlaps.some(Inverse(obo.BFO_0000108).some(Core.MaterialProduct))
<class 'owlready2.util.CallbackList'> [obo.BFO_0000008, Core.temporallyOverlaps.some(Inverse(obo.BFO_0000108).some(Core.MaterialProduct))]
<class 'owlready2.entity.ThingClass'> obo.BFO_0000008


In [317]:
axiom = iofonto["GainOfRole"].equivalent_to[0]
print(axiom)
l = 1

print(axiom.Classes[l])
print(axiom.Classes[l].Classes)
print(axiom.Classes[l].Classes[0])
print(axiom.Classes[l].Classes[0].value.Classes[1])
print(axiom.Classes[l].Classes[0].value.Classes[1].Classes[0])
print(axiom.Classes[l].Classes[0].value.Classes[1].Classes[0].value)
print(axiom.Classes[l].Classes[0].value.Classes[1].Classes[0].value.Classes[1])

check =axiom.Classes[l].Classes[0].value.Classes[1].Classes[0].value.Classes[1].property
check.property.name




obo.BFO_0000015 & (obo.BFO_0000199.some(obo.BFO_0000202 & (Core.meets.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyOverlaps.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyStarts.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)))) & Core.hasOutput.some(obo.BFO_0000023)) & obo.BFO_0000167.some(obo.BFO_0000004 & Not(obo.BFO_0000006))
obo.BFO_0000199.some(obo.BFO_0000202 & (Core.meets.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyOverlaps.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyStarts.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)))) & Core.hasOutput.some(obo.BFO_0000023)
[obo.BFO_0000199.some(obo.BFO_0000202 & (Core.meets.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.BFO_0000023)) | Core.temporallyOverlaps.some(obo.BFO_0000202 & Inverse(obo.BFO_0000108).some(obo.B

'BFO_0000108'

In [312]:
a = iofonto["MaterialArtifact"].equivalent_to[0]
print(a)
print(a.Classes[1].property)
print(a.Classes[1].property.name)



obo.BFO_0000030 & obo.BFO_0000196.some(Core.DesignedFunction)
obo.BFO_0000196
BFO_0000196


In [248]:
print(axiom.Classes[l].value.Classes[2])
isinstance(axiom.Classes[l].value.Classes[2], owlready2.class_construct.And)

obo.BFO_0000004 & Not(obo.BFO_0000006)


True

In [ ]:
owlready2.class_construct.And

In [ ]:
class_list = []

while isinstance(axiom.Classes[2].value.Classes[0], Restriction):
    print('Restriction')
    print(axiom.Classes[2].value.Classes[0].property)
    print(axiom.Classes[2].value.Classes[0].value)
    class_list.append(axiom.Classes[2].value.Classes[0].value)
    axiom = axiom.Classes[2].value.Classes[0].value.equivalent_to[0]
    print(axiom.Classes[0])
    print(axiom.Classes[1])
    print(axiom.Classes[2])

In [267]:
a = iofonto["GainofRole"]
print("none" if a is None else a)
print(dir(a))
print(a.equivalent_to)

none
['__bool__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


AttributeError: 'NoneType' object has no attribute 'equivalent_to'

In [ ]:
#Axiom to PUML
from owlready2 import *

def axiom_to_puml(class_entity):
    bfo = get_namespace("http://purl.obolibrary.org/obo/")
    iofcore = get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
    """Converts an OWL equivalent class axiom to PUML notation."""
    puml_output = []
    puml_output.append("!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml")
    class_map = {}
    counter = 1

    def get_class_name(entity):
        """Extracts and maps class names with proper prefixes."""
        nonlocal counter
        if isinstance(entity, ThingClass):  # Direct OWL class
            print(counter, 'processing', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            class_name = entity.name
            if class_name not in class_map:
                class_map[class_name] = f"c{counter}"
                puml_output.append(f'class({class_map[class_name]}, {prefix}{to_camel_case(bfo[entity.name].label[0]) if class_name.startswith("BFO") else class_name})')
                counter += 1
            print(puml_output)
            return class_map[class_name]

        elif isinstance(entity, Restriction):  # Some restriction case
            if
            print(counter)
            print('Restriction', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = prefix + to_camel_case(bfo[entity.property.name].label[0]) if entity.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
            print(prop_name)
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, And):  # Some restriction case
            print('And', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Or):  # Some restriction case
            print('Or', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Not):
            return 
        
        elif isinstance(entity, Inverse):
            print('Inverse', entity.property)
            return

        else:
            print('error',entity)
            raise ValueError(f"Unsupported entity type: {type(entity)}")

    def process_restriction(restriction, prop_name):
        """Handles OWL 'some' restrictions including unions inside."""
        nonlocal counter
        if isinstance(restriction, And):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)  # Properly escapes quotes
            puml_output.append(f"intersection({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        
        elif isinstance(restriction, Or):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)  # Properly escapes quotes
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        

        elif isinstance(restriction.value, Or):
            print('or', restriction.value)
            entities = [get_class_name(e) for e in restriction.value.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)  # Properly escapes quotes
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name

        else:  
            print('restriction else', restriction)
            class_name = get_class_name(restriction.value)
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {class_name})")
            counter += 1
            print(puml_output)
            return var_name
    
    if len(class_entity.equivalent_to) == 0:
        print('class_entity.equivalent_to',class_entity.equivalent_to)
        return
        
    else:
        print(class_entity.equivalent_to, type(class_entity.equivalent_to[0]))
        axiom = class_entity.equivalent_to[0]
        main_class_name = get_class_name(class_entity)
        processed_parts = [get_class_name(part) for part in axiom.Classes]
        print(processed_parts)

        if len(processed_parts) > 1:
            final_union = f"ce{counter}"
            entity_list = ", ".join(f'"{e}"' for e in processed_parts)
            puml_output.append(f"union({final_union}, '[{entity_list}]')")
            puml_output.append(f"equivalent({main_class_name}, {final_union})")

        return "\n".join(puml_output)

# Example Usage: [ProcuringBusinessProcess,MaterialArtifact, PieceOfEquipment], [MeasurementInformationContentEntity]
input = "GainOfRole"
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
# axiom_result = axiom_to_puml(iofonto["ProcuringBusinessProcess"])
axiom_result = axiom_to_puml(iofonto[input])
print(axiom_result)


[]